In [3]:
from datasets import load_dataset

ds = load_dataset("cnamuangtoun/resume-job-description-fit")

c:\Users\HP\Desktop\Resume Scanner\venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\datasets--cnamuangtoun--resume-job-description-fit. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating test split: 100%|██████████| 1759/1759 [00:00<00:00, 5057.97 examples/s]


In [7]:
ds['train'][120:122]

{'resume_text': ["SummaryCoordinated Maintenance Electrician at a large flatline plastic company and printed product, that is skilled in repairing, testing, maintaining and installing electrical components. Proficient in reading blueprints and schematics. Tech savvy with ability to demonstrate work experience. Multiple years of experience in autocad programs and data driven enthusiast to analyze, diagnose the process of equipment to solve technical challenges and implement solutions. Constantly using data management process to improve and assist leadership skills to train my employees. Experienced supervisor, offering proven safety record, performance-oriented nature and strong history of accomplishment in a multitude of environments. Background training and mentoring team members to optimize performance and improve quality. Reliable, industrious and diplomatic in resolving concerns. Experience installing equipment and completing work in accordance with NEC codes. Committed job seeker 